In [1]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "openaq" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:  
    print(table.table_id)

Using Kaggle's public dataset BigQuery integration.
global_air_quality


In [2]:
# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=10).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
5,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,co,840.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25
6,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,no2,166.55,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25
7,BWSSB Kadabesanahalli,Bengaluru,IN,o3,17.11,2017-02-12 01:45:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25
8,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,pm25,40.94,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25
9,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,so2,6.63,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25


In [3]:
table.schema

[SchemaField('location', 'STRING', 'NULLABLE', 'Location where data was measured', ()),
 SchemaField('city', 'STRING', 'NULLABLE', 'City containing location', ()),
 SchemaField('country', 'STRING', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', ()),
 SchemaField('pollutant', 'STRING', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'Latest measured value for the pollutant', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', ()),
 SchemaField('unit', 'STRING', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', ()),
 SchemaField('source_name', 'STRING', 'NULLABLE', 'Name of the source of the data', ()),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', ()),
 SchemaField('longitude', 'FLOAT', 'NULLABLE', 'Longitude in d

In [4]:
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [5]:
# Set up the query
query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

In [6]:
us_cities

,city
0,Houston
1,Houston
2,Houston
3,Houston
4,Houston
5,Houston
6,Houston
7,Houston
8,Houston
9,Houston


In [7]:
# What five cities have the most measurements?
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     87
Houston                                     82
Los Angeles-Long Beach-Santa Ana            63
New York-Northern New Jersey-Long Island    60
Riverside-San Bernardino-Ontario            59
Name: city, dtype: int64

In [8]:
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

# Set up the query
query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

us_cities.head()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,Freeport South Avenue I C1012,Houston,US,so2,0.0001,2016-03-06 18:00:00+00:00,ppm,Texas,28.964430,-95.354830,1.0
1,Lake Jackson C1016,Houston,US,no2,-0.0004,2016-03-06 18:00:00+00:00,ppm,Texas,29.043758,-95.472946,1.0
2,Danciger C618,Houston,US,no2,0.0005,2016-03-06 17:00:00+00:00,ppm,Texas,29.144290,-95.756600,1.0
3,Danciger C618,Houston,US,o3,0.0460,2016-03-06 17:00:00+00:00,ppm,Texas,29.144290,-95.756600,1.0
4,Galveston 99th St. C1034/A320/X183,Houston,US,no2,0.0017,2016-03-06 19:00:00+00:00,ppm,Texas,29.254473,-94.861290,1.0


In [9]:
# Query to get the score column from every row where the type column has value "job"
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country LIKE "I%" 
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 2394379 bytes.


In [10]:
# Only run the query if it's less than 100 MB
ONE_HUNDRED_MB = 100*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_HUNDRED_MB)

# Set up the query (will only run if it's less than 100 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
5,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,co,840.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.697270,0.25
6,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,no2,166.55,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.697270,0.25
7,BWSSB Kadabesanahalli,Bengaluru,IN,o3,17.11,2017-02-12 01:45:00+00:00,µg/m³,CPCB,12.938906,77.697270,0.25
8,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,pm25,40.94,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.697270,0.25
9,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,so2,6.63,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.697270,0.25


In [11]:
# returns pollutants and a new column if they have number at the end
query = """
        SELECT pollutant,
            REGEXP_CONTAINS(pollutant, r'[0-9]+') AS has_number    
        FROM `bigquery-public-data.openaq.global_air_quality`
        """

# Set up the query
query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

us_cities.head()

,pollutant,has_number
0,co,False
1,no2,True
2,o3,True
3,pm25,True
4,so2,True


In [12]:
# returns country and a new coulmn with if they starts with S
query = """
        SELECT country,
            REGEXP_CONTAINS(country, r'^[Ss].*') AS starts_with_s 
        FROM `bigquery-public-data.openaq.global_air_quality`
        """

# Set up the query
query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

us_cities.head()

,country,starts_with_s
0,IN,False
1,IN,False
2,IN,False
3,IN,False
4,IN,False
